In [ ]:
import pandas as pd
import numpy as np
from mip import *
from pulp import *
import csv
from pulp import GLPK
import matplotlib.pyplot as plt
import math
import statistics
from pytictoc import TicToc
import time
from ttictoc import tic,toc
import requests
import json
import polyline
import folium

In [ ]:
# Supplier profile
Supplier_profile = pd.read_csv("after-addition monthly suppliers profile.csv")
display(Supplier_profile)

In [ ]:
# Suppliers profile

B = 17 # set of biomass
S = 77 # set of suppliers
T = 12 # set of time-step
Supplier_Profile=np.zeros([B,S,T])
for s in range(S):
    for b in range(B):
        for t in range(T):
                Supplier_Profile[b,s,t] = Supplier_profile.iloc[(17*s)+b,t+2]

In [ ]:
# Power plant profile
PowerPlant_profile = pd.read_csv("plants profile.csv")
PowerPlant_profile

In [ ]:
# Distance
Distance = pd.read_csv("distance.csv")
display(Distance)

In [ ]:
# Biomass data
Biomass_data = pd.read_csv("biomass.csv")
display(Biomass_data)

In [ ]:
num_days_in_month = np.array([31,28,31,30,31,30,31,31,30,31,30,31])

In [ ]:
# Distance new plants
LPtP= pd.read_csv("industrial area loc.csv")
LPtP=LPtP.drop(['From/To'], axis=1)
display(LPtP)

# ==========================================================
# Optimize with the lowest sum distance (center of gravity)
# ==========================================================

In [ ]:
tic()
C_TF = 455542.86 # Fixed transportation cost per a truck
D_Eth = 4790000 #L/day
D_BElc = 3940 # MW (target at 3940 MW)
D_GElc = 387 # MW (target at 387 MW)
C_IF = 0 #THB/ton

# PuLP
# Set==========================================================================
B = 17 # set of biomass
S = 77 # set of suppliers
PQ123 = 215 # set of plant
PQ4 = 185
PQ5 = 27
T = 12 # set of time period
F = 77
M = 2 # new plants

prob = LpProblem(name = 'Additional_Biomass_improved_SC', sense = LpMinimize)

Ynew = LpVariable.dicts("Binary_variables",((m,f) for m in range(M) for f in range(F)), lowBound=0, cat="Binary")

# Objective ==========================================================================


prob+= lpSum(LPtP.iloc[s,f]*Ynew[m,f] for m in range(M) for s in range(S) for f in range(F))

for m in range(M):
    prob+=Ynew[m,0] == 0

prob+= lpSum(Ynew[m,f] for m in range(M) for f in range(F)) == M    
    
status=prob.solve() 

elapsed = toc()
print('Elapsed time:',elapsed)

print(LpStatus[status])
print('Obj value:   ', value(prob.objective))

for f in range(F):
    for m in range(M):
        if Ynew[m,f].varValue == 1:
            print('Y_{}_{}  :  '.format(m+1,f+1), Ynew[m,f].varValue)
            
            
for m in range(M):
    print("Total distance from all suppliers to plant {} is   ".format(m+1), lpSum(LPtP.iloc[s,f]*Ynew[m,f].varValue for s in range(S) for f in range(F)))